In [ ]:
%cd ..
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import imageio
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example dataset and example kernels

In [ ]:
# images = tfds.load(name="bsds500", split="validation") \
#                 .map(datasets.get_image) \
#                 .map(datasets.to_float32) \
#                 .map(datasets.from_255_to_1_range)
images = tfds.load(name="set5", split="test") \
                .map(datasets.get_value('hr')) \
                .map(datasets.to_float32) \
                .map(datasets.from_255_to_1_range)

In [ ]:
noise_stddev = 0.35
image = datasets.get_one_example(images, index=1)[None,...]
visualize.draw_images(image)

# Degrade the image

In [ ]:
degraded = image + tf.random.normal(tf.shape(image), 0, noise_stddev)

In [ ]:
degraded_psnr = dppp.psnr(image, degraded).numpy()[0]
degraded_fsim = dppp.fsim(image, degraded).numpy()[0]
print(f"Degraded PSNR: {degraded_psnr:0.2f}, FSIM: {degraded_fsim:0.4f}")

# Run different Methods and Models

In [ ]:
models = [
#     'dae_0.04314',
#     'drcnn_0.05',
    'drunet+_0.0-0.2',
    'drugan+_0.0-0.2',
]

In [ ]:
all_reconstructed = []
all_psnr = []
all_fsim = []

for model_name in models:
    model_path = os.path.join('models', f'{model_name}.h5')
    denoiser, _ = dppp.load_denoiser(model_path)
        
    print(f"Running {model_name}...")
    rec = denoiser(degraded, noise_stddev)
    
    psnr = dppp.psnr(image, rec).numpy()[0]
    fsim = dppp.fsim(image, rec).numpy()[0]
    print(f"Reconstructed PSNR: {psnr:0.2f}, FSIM: {fsim:0.4f}")
    
    all_reconstructed.append(rec)
    all_psnr.append(psnr)
    all_fsim.append(fsim)

# Visualize

In [ ]:
visualize.draw_images([image[0], degraded[0], *[x[0] for x in all_reconstructed]], ncol=4, figsize=(50, 40))

# Export

In [ ]:
export_dir = os.path.join('visualize', 'denoise')

In [ ]:
os.makedirs(export_dir)

def export_path(x):
    return os.path.join(export_dir, x)

def to_uint8(x):
    return tf.cast(tf.clip_by_value(x, 0, 1) * 255, tf.uint8)

def write_to(file_name, img, psnr, fsim):
    imageio.imwrite(export_path(f'{file_name}.png'), to_uint8(img[0]))
    with open(export_path(f'{file_name}.txt'), 'w') as f:
        f.write(f"PSNR: {psnr}, FSIM: {fsim}")

#### Images

# Ground truth
imageio.imwrite(export_path('gt.png'), to_uint8(image[0]))

# Degraded
write_to('degraded', degraded, degraded_psnr, degraded_fsim)

# Reconstructed
for model_name, rec, psnr, fsim in zip(models, all_reconstructed, all_psnr, all_fsim):
    write_to(f'{model_name}', rec, psnr, fsim)

# Highlight a Region of Interest

Also add results from other methods before running these cells

In [ ]:
import os
import imageio
from highlight import highlight, highlight_and_save

### Preview and Region Selection

In [ ]:
image = imageio.imread(os.path.join(export_dir, 'gt.png'))

# Location of the region of interest
center = (135,215)
extent = 25

# Size of the box
rel_width = 0.5

# Positioning of the box
border = 0.01
top = False
right = False
label_border = 0.09
fontsize=12

fig = highlight(image, center, extent, rel_width, top, right, border, label_border, fontsize)

### Highlight

In [ ]:
highlight_and_save(export_dir, center, extent, rel_width, top, right, border, label_border, fontsize)